# Benzinga-Nachrichten-Verarbeitung

Don't forget to shut down cluster at the end of the day!!!

## Setup up

### System settings

In [3]:
using_colab = False
import os

In [4]:
%%capture
if using_colab:

  from google.colab import drive
  drive.mount('/content/drive')

  cwd = "/content/drive/MyDrive/NewsTrading/trading_bot"
  %cd /content/drive/MyDrive/NewsTrading/trading_bot
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/MyDrive/extreme-lore-398917-ac46de419eb2.json"


  !pip install dask-cloudprovider
else:
  cwd = 'G:/Meine Ablage/NewsTrading/trading_bot'
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="G:/Meine Ablage/extreme-lore-398917-ac46de419eb2.json"


### CUDF für hardware acceleration

In [5]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
# import cudf

### Google cloud

In [6]:
from dask_cloudprovider.gcp import GCPCluster
from dask.distributed import Client

In [7]:
os.environ["DASK_CLOUDPROVIDER__GCP__PROJECTID"]="extreme-lore-398917"

In [28]:
# Can take up to 5 minutes
# with n_workers=2, creates 2
cluster = GCPCluster(
    n_workers=1,
    zone='us-east1-c',
    machine_type='n1-highcpu-8',
    projectid="extreme-lore-398917",
    ngpus=0
)

Launching cluster with the following configuration: 
  Source Image: projects/ubuntu-os-cloud/global/images/ubuntu-minimal-1804-bionic-v20201014 
  Docker Image: daskdev/dask:latest 
  Machine Type: n1-highcpu-8 
  Filesystem Size: 50 
  Disk Type: pd-standard 
  N-GPU Type:  
  Zone: us-east1-c 
Creating scheduler instance
dask-bd6ac445-scheduler
	Internal IP: 10.142.0.41
	External IP: 104.196.65.162
Waiting for scheduler to run at 104.196.65.162:8786
Scheduler is running


C:\Users\Adria\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources. Hang tight! 
  next(self.gen)


Creating worker instance
dask-bd6ac445-worker-a44b6cfe
	Internal IP: 10.142.0.42
	External IP: 34.138.120.240


In [29]:
client = Client(cluster)

g:\Meine Ablage\NewsTrading\trading_bot\.venv\Lib\site-packages\distributed\client.py:1391: VersionMismatchWarning: Mismatched versions found

+---------+----------------+-----------------+---------+
| Package | Client         | Scheduler       | Workers |
+---------+----------------+-----------------+---------+
| python  | 3.11.3.final.0 | 3.10.12.final.0 | None    |
+---------+----------------+-----------------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [30]:
# This is asynchronously set up. Might not show workers immediately!!!
client

Connection method: Cluster object,Cluster type: dask_cloudprovider.GCPCluster
Dashboard: http://104.196.65.162:8787/status,
Dashboard: http://104.196.65.162:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tls://10.142.0.41:8786,Workers: 0
Dashboard: http://10.142.0.41:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [31]:
client.upload_file("requirements.txt")
def install():
    commands = \
    ["import os",
    "sudo apt update",
    "sudo apt install maven",
    "mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec('sutime').origin).parent / 'pom.xml')')",
    "py -m venv myenv",
    ".\myenv\Scripts\activate",
    "pip install -r requirments"]
    for cmd in commands:
        os.system(cmd)

client.run(install)  # Run on all workers

{}

In [12]:
# from dask.distributed import PipInstall

# plugin = PipInstall(packages=["html2text",
#                                 "datefinder",
#                                 "dask[complete]",
#                                 "nltk",
#                                 "dateparser",
#                                 "sutime",
#                                 "yfinance"],
#                     pip_options=["--upgrade"])
# client.register_plugin(plugin)

In [46]:
cluster.close()
client.close()

Closing Instance: dask-bd6ac445-worker-a44b6cfe
Closing Instance: dask-bd6ac445-scheduler


### Imports

In [ ]:
%%capture
if using_colab:
  !sudo apt update
  !sudo apt install maven;

  !pip install pyarrow==11.0.0
  !pip install html2text
  !pip install datefinder
  !pip install -U dask[complete]
  !pip install nltk
  !pip install dateparser
  !pip install pyngrok
  !pip install sutime
  !pip install pyngrok

  # This is required for sutime
  !mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")');

In [13]:
%load_ext autoreload
%autoreload 2
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from src.preprocessing.news_parser import get_company_abbreviation, yahoo_get_wrapper, \
                                          infer_author
from pyngrok import ngrok, conf
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
# TODO: Das hier muss auch noch auf den clients installiert werden

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Adria\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Adria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Setup NGROK tunnel

In [7]:
# ------- Only required when working on google colab, not required when working with google cloud engine -----------------
if using_colab:
  conf.get_default().auth_token = "2WntwErWDt9LxQ2Jfp6C8OxDAMK_7iZVdC1utyZET1PE8cuUg"

  public_url = ngrok.connect(8787).public_url
  print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, 8787))

 * ngrok tunnel "https://d6ff-34-142-214-64.ngrok-free.app" -> "http://127.0.0.1:8787"


In [8]:
if 'client' not in globals():
  !python -m pip install jupyter-server-proxy
  client = Client()

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:34601
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34465'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42561'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40409'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46137'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42305', name: 3, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42305
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:33790
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45561', name: 2, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute str

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/unraw2_bzg/'
output_dir = cwd+'/data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

,author,inferred_author
Benzinga,1061214,NaN
PRNewswire,305720,587242.0
Globe Newswire,293466,475171.0
Business Wire,268561,293052.0
Newsfile,70877,NaN
ACCESSWIRE,62615,81054.0
"AB Digital, Inc.",9936,NaN
WebWire,6404,NaN
PRWeb,2617,NaN
News Direct,2080,NaN


In [ ]:
auhtor_value_counts.sum().diff()

author                  NaN
inferred_author   -646971.0
dtype: float64

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

PRNewswire        587242
Globe Newswire    475171
Business Wire     293052
ACCESSWIRE         81054
Name: inferred_author, dtype: Int64

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

1436519

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Globe Newswire    44589
PRNewswire        31440
ACCESSWIRE        16434
Name: inferred_author, dtype: Int64

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Vorgehen

Benötigt:
---------
- data/tickers.pkl (alt) \\
- data_shared/corporation_endings.txt \\
- input_dir = data/unraw3_bzg/ \\
- output_dir = data/unraw4_bzg/

Produziert:
-----------
- data_shared/all_ticker_name_mapper.parquet

--------------

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt! Bzw. generell ist das eine gute Datensäuberungs-Maßnahme.

In [ ]:
input_dir = cwd+'/data/unraw3_bzg/'
output_dir = cwd+'/data/unraw4_bzg/'
ddf = dd.read_parquet(input_dir)

In [ ]:
all_tickers = ddf.stocks.unique().compute()

### Full-Name-Discovery

In [ ]:
company_names = all_tickers.apply(lambda x: yahoo_get_wrapper(x))

In [ ]:
all_mapper = pd.concat([all_tickers, company_names], axis=1)

In [ ]:
all_mapper.columns = ["ticker", "company_name"]

In [ ]:
all_mapper = all_mapper.dropna() # This drops like half of the tickers.

In [ ]:
all_mapper[all_mapper.company_name.apply(lambda x: "Alphabet" in x)]

,ticker,company_name
653,GOOG,Alphabet Inc.
10617,GOOGL,Alphabet Inc.


In [ ]:
print(len(all_mapper))
print(len(all_mapper.company_name.unique()))

11383
11072


In [ ]:
vcs = all_mapper.company_name.value_counts()
vcs = vcs[vcs >= 2]

In [ ]:
# Ticker-Grouping
all_mapper[all_mapper.company_name.isin(vcs.index)].sort_values("company_name")

,ticker,company_name
21729,VCXA,10X Capital Venture Acquisition Corp. II
19821,VCXAU,10X Capital Venture Acquisition Corp. II
20368,ADER,26 Capital Acquisition Corp.
18164,ADERU,26 Capital Acquisition Corp.
18032,VIIAU,7GC & Co. Holdings Inc.
...,...,...
15333,ICCT,iCoreConnect Inc.
21288,IONR,ioneer Ltd
20748,GSCCF,ioneer Ltd
18265,OSSFF,Össur hf.


Es ist nicht leicht zu sagen, welchen von den Tickern wir bevorzugen sollten. Abgleichen mit den Aktientickern des Kursdatensatzes notwendig, um zu sehen, ob überhaupt nur ein Ticker übereinstimmt. Wenn es für beide Ticker eine Kurszeitreihe gibt, dann sollten wir die nehmen, die ein höheres historisches Volumen hat. Dies ist allerdings etwas, was wir später machen und nicht jetzt. Hier wollen wir zunächst nur die Nachrichten verarbeiten, weswegen wir nur die NaN-Unternehmen rausnehmen und den Rest - *ohne Ticker-Filtering* - weiterverarbeiten.

In [ ]:
mapper = all_mapper

In [ ]:
mapper.columns = ["ticker", "company_names"]
mapper = mapper[mapper.isna().sum(axis=1) == 0]

In [ ]:
mapper = mapper.set_index("ticker")

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
# Apply get_company_abbreviation twice in order to get rid of Enterprise, Ltd.
# Otherwise , Ltd. remains.
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x, company_endings=company_endings))

In [ ]:
print(mapper.short_name.isna().sum()) # 2037 stocks for which we don't have an ending to abbreviate
# mapper.loc[:, "short_name"] = mapper.short_name.fillna(mapper.company_names)
mapper = mapper.applymap(lambda x: x.strip(" "))

0


In [ ]:
mapper.to_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
mapper = pd.read_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [ ]:
ddf.shape[0].compute()

1436519

In [ ]:
filt_ddf.shape[0].compute()

1017673

Es verbleiben circa 1 Mio. Nachrichten, für die wir den Ticker zu einem FIrmennamen auflösen können.

Diese Nachrichten können wir nun wirklich parsen, und danach ordentlich kategorisieren.

In [ ]:
ddf = filt_ddf
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# TODO: Kann effizienter werden mit GroupBy-Operation
ddf["company_name"] = ddf.stocks.apply(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.apply(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest/', name_function=name_function)

### Firmennamen-Nachrichtenkörper-Verifikation

In [40]:
ddf = dd.read_parquet(cwd+'/data/latest/')

In [34]:
# ddf["time"] = ddf["time"].astype(str)

In [41]:
def f(df):
    df["time"] = df["time"].astype(str)
    return df
# futures = client.submit(f, ddf)
# ddf = ddf.map_partitions(f)

In [45]:
# results = client.gather(futures)

RuntimeError: Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments


In [36]:
mask = ddf.apply(lambda x: bool(re.search(x["short_name"], x["body"].replace("\n", " "), re.IGNORECASE)) or \
                 bool(re.search(x["short_name"], x.title, re.IGNORECASE)),
                 axis=1,
                 meta=pd.Series(dtype=bool))

In [ ]:
# Around 11k stocks before `filtering`
# len(ddf.stocks.unique())

# Around 10k stocks after `filtering`
# len(ddf[mask].stocks.unique())

In [ ]:
ddf[mask].shape[0].compute()

897403

In [ ]:
# Case 1: Firmenname wird in abgekürzter Version benutzt, die wir nicht methodisch rekonstruieren können
#   Z.B.: IBM -> International Business Machines,
#   oder UPS -> United Parcel Service
#   oder GE -> General Electric
# Case 2: Ticker wurde recycled und zeigt inzwischen auf eine andere Firma.
# Will hierzu nicht auch noch Daten kaufen, deswegen werden hier leider einige Nachrichten verworfen werden.

ddf[~mask].shape[0].compute() # ~120k Nachrichten mit `FALSCHEM` Firmennamen -> Textinhalt ableichen mit Firmennamen des Kursdaten-Datensatzes.

120270

In [ ]:
# ddf[~mask].stocks.value_counts().compute().head(15)

In [37]:
# Filter for high conviction entries
ddf = ddf[mask]

In [38]:
ddf = ddf.repartition(npartitions=30)

In [39]:
ddf.head()

RuntimeError: Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments


### Duplikate Entfernen

In [ ]:
# import cudf

In [ ]:
# ddf = ddf.map_partitions(cudf.from_pandas)

In [16]:
res = ddf.map_partitions(lambda x: x.drop_duplicates())

In [17]:
# res.shape[0].compute()

In [18]:
ddf = res

In [19]:
ddf.head()

INFO:distributed.core:Connection to tcp://127.0.0.1:46800 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:35481', name: 0, status: running, memory: 0, processing: 1> (stimulus_id='handle-worker-cleanup-1697636819.290386')
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:36497'. Reason: worker-handle-scheduler-connection-broken
INFO:distributed.core:Connection to tcp://127.0.0.1:46810 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:37471', name: 1, status: running, memory: 0, processing: 1> (stimulus_id='handle-worker-cleanup-1697636820.4134212')
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:37775'. Reason: worker-handle-scheduler-connection-broken
INFO:distributed.core:Connection to tcp://127.0.0.1:46824 has been closed.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:36961', name: 2, status: running, memory: 0, processing: 1> (stimulus_id='han

KilledWorker: ignored

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest1/', name_function=name_function)

#### Convert ddf to pd.DataFrame

In [ ]:
#ddf = ddf.compute()
ddf = ddf.sort_values("time")

### Timedeltas zwischen Nachrichtenmeldungen



Wir sehen, dass einige Nachrichten dupliziert vorkommen, d.h. mit einem Timedelta von 0 und mit derselben Überschrift etc. diese gilt es zu eliminieren.

In [ ]:
tmp = ddf[["time", "stocks"]]

In [ ]:
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())

AttributeError: ignored

In [ ]:
# ~3 minutes evaluates to true
# (news_timedeltas.index == ddf.index).all()

True

In [ ]:
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

<ipython-input-227-d78fa882f109>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))


In [ ]:
news_timedeltas = ddf.timedelta

In [ ]:
news_timedeltas.iloc[0].components

Components(days=100, hours=0, minutes=0, seconds=0, milliseconds=0, microseconds=0, nanoseconds=0)

In [ ]:
same_day_timedeltas = news_timedeltas.apply(lambda x: x.components.days == 0)

In [ ]:
(same_day_timedeltas == 0).sum()

714182

In [ ]:
same_hour_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0))

In [ ]:
print(same_hour_timedeltas.sum())

43341


In [ ]:
same_minute_timedeltas = news_timedeltas.apply(lambda x: (x.components.days == 0) & \
                                               (x.components.hours == 0) & \
                                               (x.components.minutes == 0))

In [ ]:
print(same_minute_timedeltas.sum())

16357


In [ ]:
same_day_ddf = ddf.loc[same_day_timedeltas]

In [ ]:
ddf.stocks.value_counts().describe()

count    9939.000000
mean       89.795955
std       163.034946
min         1.000000
25%         4.000000
50%        30.000000
75%       116.000000
max      4984.000000
Name: stocks, dtype: float64

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert.

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Topic Modeling (Nach baschluss Auslagern in Python file)

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag
import numpy as np
from typing import List

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Remove numbers
docs = ddf.body.apply(lambda x: re.sub(r'\d+', '', x), meta=pd.Series(dtype="string"))

In [ ]:
def lemmatize_words(string_list: List[str], wnl: WordNetLemmatizer):
  return [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(string_list)]

In [ ]:
def tokenize(doc):
    wnl = WordNetLemmatizer()

    # Tokenize document
    x = str.lower(doc)
    tokens = word_tokenize(x)

    # Remove words that are only one character.
    # Lemmatize the documents.
    lemmatized_tokens = np.array(lemmatize_words(tokens, wnl))
    return lemmatized_tokens

### TF-IDF

In [ ]:
def keyword_filter(tokens, keyword_list):
  # Tokens here are assumed to be lemmatized
  is_keyword_mask = np.vectorize(lambda x: x in keyword_list)(tokens)
  relevant_tokens = tokens[is_keyword_mask.astype(bool)]
  return relevant_tokens

In [ ]:
wnl = WordNetLemmatizer()
keywords_list1 = ["launch", "business", "strategy", "management", "product", "service", "app", "customer", "merge"]
keywords_list2 = ["upgrade", "downgrade", "raise", "cut", "buy", "sell", "hold", "outperform", "underperform", "analyst", "estimate"]
keywords_list3 = ["ebit", "eps", "earnings", "report", "financial", "quarter", "annual", "year", "ended", "net", "income"]
total_keywords = keywords_list1 + keywords_list2 + keywords_list3
len(total_keywords)


31

In [ ]:
# Tokenization + rough filtering
tfidf_docs = docs.apply(lambda x: keyword_filter(tokenize(x), total_keywords), meta=pd.Series(dtype="object"))

In [ ]:
tfidf_docs = tfidf_docs.compute()

In [ ]:
keywords1 = lambda x: keyword_filter(x, keywords_list1)
keywords2 = lambda x: keyword_filter(x, keywords_list2)
keywords3 = lambda x: keyword_filter(x, keywords_list3)

In [ ]:
# custom tokenizers
vectorizer = TfidfVectorizer(tokenizer=keywords1)
output_corpus = vectorizer.fit_transform(tfidf_docs)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-e0c7ea44d653>", line 3, in <cell line: 3>
    output_corpus = vectorizer.fit_transform(tfidf_docs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 2133, in fit_transform
    X = super().fit_transform(raw_documents)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 1388, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents, self.fixed_vocabulary_)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 1275, in _count_vocab
    for feature in analyze(doc):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py", line 111, in _analyze
    doc = preprocessor(doc)
  File "/usr/local/lib/python3.10/dist-packages/skle

In [ ]:
# custom tokenizers
combined_features = FeatureUnion([
    ('kwd1', TfidfVectorizer(tokenizer=keywords1)),
    ('kwd2', TfidfVectorizer(tokenizer=keywords2)),
    ('kwd3', TfidfVectorizer(tokenizer=keywords3))
])

pipeline = Pipeline([('bow', combined_features)])
output_corpus = pipeline.fit_transform(tfidf_docs)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-2149289da8c1>", line 9, in <cell line: 9>
    output_corpus = pipeline.fit_transform(tfidf_docs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 445, in fit_transform
    return last_step.fit_transform(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 1192, in fit_transform
    results = self._parallel_func(X, y, fit_params, _fit_transform_one)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 1214, in _parallel_func
    return Parallel(n_jobs=self.n_jobs)(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-2149289da8c1>", line 9, in <cell line: 9>
    output_corpus = pipeline.fit_transform(tfidf_docs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 445, in fit_transform
    return last_step.fit_transform(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py", line 140, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 1192, in fit_transform
    results = self._parallel_func(X, y, fit_params, _fit_transform_one)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 1214, in _parallel_func
    return Parallel(n_jobs=self.n_jobs)(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

In [ ]:
assert output_corpus.shape[1] == total_keywords

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-0d1ed5b1a83b>", line 1, in <cell line: 1>
    assert output_corpus.shape[1] == total_keywords
NameError: name 'output_corpus' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3

In [ ]:
import numpy as np
def  f(x):
  return x[0, :9].sum(), x[0, 9:20].sum(), x[0, 20:].sum()
b = np.apply_along_axis(f, 1, output_corpus.todense())

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-1c14677b52bf>", line 4, in <cell line: 4>
    b = np.apply_along_axis(f, 1, output_corpus.todense())
NameError: name 'output_corpus' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-1c14677b52bf>", line 4, in <cell line: 4>
    b = np.apply_along_axis(f, 1, output_corpus.todense())
NameError: name 'output_corpus' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/usr/local/lib/python3.1

In [ ]:
b_df = pd.DataFrame(b)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-eb351e4d673c>", line 1, in <cell line: 1>
    b_df = pd.DataFrame(b)
NameError: name 'b' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultrat

In [ ]:
c = np.apply_along_axis(lambda x: np.argmax(x), 1, b)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-da0f5ac15543>", line 1, in <cell line: 1>
    c = np.apply_along_axis(lambda x: np.argmax(x), 1, b)
NameError: name 'b' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/di

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-da0f5ac15543>", line 1, in <cell line: 1>
    c = np.apply_along_axis(lambda x: np.argmax(x), 1, b)
NameError: name 'b' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
pd.Series(c).value_counts()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-ba5c9c6d2bf9>", line 1, in <cell line: 1>
    pd.Series(c).value_counts()
NameError: name 'c' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/u

### LDA

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import Phrases
import nltk
nltk.download('wordnet')

In [ ]:
# Split the documents into tokens.
lda_docs = docs.apply(lambda x: word_tokenize(str.lower(x)), meta=pd.Series(dtype="object"))

In [ ]:
# Remove words that are only one character.
# Lemmatize the documents.
lemmatizer = WordNetLemmatizer()
lda_docs = lda_docs.apply(lambda x: [lemmatizer.lemmatize(token) for token in x if len(token) >1], meta=pd.Series(dtype="object"))

In [ ]:
lda_docs.head()

0    [new, york, jan., /prnewswire-firstcall/, --, ...
1    [boise, idaho, jan., /prnewswire-firstcall/, -...
2    [salt, lake, city, jan., /prnewswire-firstcall...
3    [stockholm, --, business, wire, --, regulatory...
4    [cleveland, jan., /prnewswire-firstcall/, --, ...
dtype: object

In [ ]:
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
def f(doc):
  doc = doc
  for token in bigram[doc]:
      if '_' in token:
          # Token is a bigram, add to document.
          doc.append(token)
  return doc
lda_docs = lda_docs.apply(lambda doc: f(doc))

INFO:distributed.core:Event loop was unresponsive in Nanny for 3.16s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.17s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.17s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 3.17s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.18s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeout

In [ ]:
# lda_docs.name = "body"
# lda_docs = lda_docs.to_frame().reset_index()
# lda_docs = lda_docs.reset_index().repartition()

In [ ]:
# lda_docs.to_parquet(cwd + "/data/lda_docs", schema={"body": pa.string(),  "index":pa.int32()})
# lda_docs = dd.read_parquet(cwd + "/data/lda_docs")

In [ ]:
# Remove rare and common tokens.

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=1000, no_above=0.6)

In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

INFO:distributed.core:Event loop was unresponsive in Nanny for 15.46s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 15.47s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 15.48s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 15.40s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 15.44s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause ti

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [ ]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

## Nachrichten-Parsing


In [ ]:
dask.config.set(scheduler="processes")

In [ ]:
sample_partition = ddf.get_partition(20)
x = sample_partition.head(5).iloc[0]
x.body

In [ ]:
filter_body(x.body, x.stocks, x.author, x.name, x.company_name, x.short_name) # x.name == time

Anwenden der filter_body-Funktion auf alle Reihen:

In [ ]:
part_n = 1
par = ddf.get_partition(part_n)

In [ ]:
par["parsed_body"] = par.apply(lambda x: filter_body(x.body,
                                                      x.stocks,
                                                      x.author,
                                                      x.name,
                                                      x.company_name,
                                                      x.short_name),
                                axis=1,
                                meta=pd.Series(dtype="string"))

In [ ]:
par = par.drop(columns=["body"])

In [ ]:
output_dir = 'data/parsed_bzg/'
name_function = lambda x: f"data-{x+part_n}.parquet"
par.to_parquet(output_dir, name_function=name_function)

In [ ]:
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)

## Analyse der durschnittlichen Tokenlänge

## Generating timestamp-stock pairs in order to know for which days we need stock data